In [2]:
:opt no-lint

In [3]:
-- Prolog term
data Term = V String        -- logic variable.       String 대문자로 시작
          | T String [Term] -- atom when empty list. String 소문자로 시작
      deriving (Eq,Ord,Show)

In [4]:
import Data.List
-- Prolog term처럼 보이게 좀더 예쁘게 출력할 수 있는 문자열로
pp (V x) = x
pp (T a []) = a
pp (T f ts) = f ++ (paren . concat . intersperse "," $ map pp ts )

paren s = "("++s++")"

In [5]:
-- 변수 X
V "X" 
pp(V "X")

V "X"

"X"

In [6]:
-- 아톰 int 
T "int" []
pp(T "int" [])

T "int" []

"int"

In [7]:
-- 복합항 f(int,int)
T "f" [T "int" [], T "int" []]
pp(T "f" [T "int" [], T "int" []])

T "f" [T "int" [],T "int" []]

"f(int,int)"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [8]:
occurs :: String -> Term -> Bool
occurs x (V x') = x==x'
occurs x (T _ ts) = any (occurs x) ts
-- occurs x (T _ ts) = or [occurs x t | t<-ts]

In [9]:
occurs "X" (T "f" [T "int" [], V "Y"])
occurs "X" (T "f" [T "int" [], V "X"])

False

True

In [10]:
subst :: (String,Term) -> Term -> Term
subst (x,t) u@(V y)  | x == y    = t
                     | otherwise = u
subst (x,t) (T f us) = T f [subst (x,t) u | u<-us]

In [11]:
-- {X |-> f(int,int)} f(X,X) ~~> f(f(int,int),f(int,int))
int = T "int" []

tm1 = T"f"[V"X",V"X"]
tm2 = subst ("X",T"f"[int,int]) (T"f"[V"X",V"X"])

pp tm1
pp tm2

"f(X,X)"

"f(f(int,int),f(int,int))"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [12]:
zip [1,2,3,4] [11,12,13,14]

[(1,11),(2,12),(3,13),(4,14)]

In [13]:
type Eqn = (Term,Term)       -- t1 = t2
type Subst = [(String,Term)] -- { X1 |-> t1, ..., Xn |-> tn }

-- 일치화 알고리듬을 위 규칙에 따라 작성
unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((T f ts, T g us): es, s)
       | f==g && length ts==length us  = unif (zip ts us ++ es, s)
unif ((V x   , V x'  ): es, s) | x==x' = unif (es, s)
unif ((V x   , t     ): es, s) | not(x `occurs` t) = unif (es', (x,t):s')
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
        s' = [(x',subst (x,t) t') | (x',t')<-s]
unif ((t     , V x   ): es, s) = unif ((V x,t):es, s)
unif (es, s) = (es, s)

In [14]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], V"Y" ]            -- c(2,Y)

unif ([(t1,t2)], [])

([],[("Y",T "c" [T "3" [],T "2" []]),("X",T "2" [])])

In [15]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], T"c"[V"X",V"Y"] ] -- c(2,c(X,X))

unif ([(t1,t2)], [])

([(T "3" [],T "2" []),(T "2" [],V "Y")],[("X",T "2" [])])

## 규칙 실행
규칙 실행은 여러 세트의 연립방정식을 만들어냄

규칙 실행의 결과 타입은 `[ [Eqn] ]`

In [16]:
-- Prolog에서 =
(.=) :: Term -> Term -> [ [Eqn] ]
t1 .= t2  =  [ [(t1,t2)] ]

-- Prolog에서 ,
(.&) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .& ess2  = [es1++es2  | es1<-ess1, es2<-ess2]

-- Prolog에서 ;
(.|) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .| ess2  = ess1 ++ ess2

### 필요에 따라서 새로운 논리변수를 만들어낼 수 있어야

Prolog 규칙을 하스켈로 옮겨 실행하기 위해 지난 번 UnifPrologTerm 노트북에서 작성한 함수의 타입은 다음과 같았다.
```haskell
likes  :: (Term, Term) -> [[Eqn]]
couple :: (Term, Term) -> [[Eqn]]
```

새로운 논리변수를 만드는 한가지 방법은 숫자를 매겨 "_0", "_1", "_2" 이런 식으로 겹치지 않게 변수를 나가는 것이다.

새로운 변수를 공급해줄 수 있는 상자에 해당하는 타입생성자를 `m`이라고 하면 대략 다음과 같은 타입이 되리라 예상해 볼 수 있다.

```haskell
father  :: Monad m => (Term, Term) -> m [ [Eqn] ]
grandpa :: Monad m => (Term, Term) -> m [ [Eqn] ]
```

In [17]:
t1 .=. t2 = return $ t1.=t2
m1 .&. m2 = (.&) <$> m1 <*> m2
m1 .|. m2 = (.|) <$> m1 <*> m2

In [18]:
:type (.=)
:type (.&)
:type (.|)

(.=) :: Term -> Term -> [[Eqn]]

(.&) :: [[Eqn]] -> [[Eqn]] -> [[Eqn]]

(.|) :: [[Eqn]] -> [[Eqn]] -> [[Eqn]]

In [19]:
:type (.=.)
:type (.&.)
:type (.|.)

(.=.) :: forall (m :: * -> *). Monad m => Term -> Term -> m [[Eqn]]

(.&.) :: forall (f :: * -> *). Applicative f => f [[Eqn]] -> f [[Eqn]] -> f [[Eqn]]

(.|.) :: forall (f :: * -> *). Applicative f => f [[Eqn]] -> f [[Eqn]] -> f [[Eqn]]

In [20]:
tim = T "tim" []
bob = T "bob" []
jane = T "jane" []
mary = T "mary" []
john = T "john" []
luke = T "luke" []
kate = T "kate" []

male(x) = (x.=.tim) .|.
          (x.=.bob) .|.
          (x.=.john)
          
female(x) = (x.=.jane) .|.
            (x.=.mary)

parent(p,c) = ( (p.=.tim) .&.(c.=.bob)  ) .|.
              ( (p.=.jane).&.(c.=.bob)  ) .|.
              ( (p.=.john).&.(c.=.tim)  ) .|.
              ( (p.=.mary).&.(c.=.tim)  ) .|.
              ( (p.=.luke).&.(c.=.john) ) .|.
              ( (p.=.kate).&.(c.=.john) )

father(f,c) = parent(f,c) .&. male(f)

In [21]:
solve ess = [unif (es,[]) | es<-ess]

In [22]:
:type male
:type female
:type parent
:type father

male :: forall (f :: * -> *). Monad f => Term -> f [[Eqn]]

female :: forall (f :: * -> *). Monad f => Term -> f [[Eqn]]

parent :: forall (f :: * -> *). Monad f => (Term, Term) -> f [[Eqn]]

father :: forall (f :: * -> *). Monad f => (Term, Term) -> f [[Eqn]]

In [23]:
import Control.Monad.Trans.State

newVar :: State Int Term
newVar = do modify (+1)
            i <- get
            return $ V('_':show (i::Int))

In [24]:
grandpa(g,y) = do p <- newVar
                  parent(p,y) .&. father(g,p)

In [25]:
:type grandpa

grandpa :: (Term, Term) -> StateT Int Identity [[Eqn]]

In [26]:
(ess,i) = runState ( father(V"x",bob) ) 0
-- ess
-- mapM_ print ess
i
result = solve ess
mapM_ print [sigma | ([],sigma) <- result]

0

[("x",T "tim" [])]

In [27]:
(ess,i) = runState ( grandpa(V"x",V"y") ) 0
-- ess
-- mapM_ print ess
i
result = solve ess
mapM_ print [sigma | ([],sigma) <- result]

1

[("x",T "john" []),("y",T "bob" []),("_1",T "tim" [])]

In [28]:
ancestor(a,x) = parent(a,x)
                .|.
                do p <- newVar
                   parent(p,x) .&. ancestor(a,p)

In [31]:
(ess,i) = runState ( ancestor(V"x",bob) ) 0
-- ess
-- mapM_ print $ take 20 ess
-- i
result = solve $ take 1000 ess
mapM_ print [sigma | ([],sigma) <- result]

[("x",T "tim" [])]
[("x",T "jane" [])]
[("x",T "john" []),("_1",T "tim" [])]
[("x",T "mary" []),("_1",T "tim" [])]